In [1]:
# pip install mysql-connector-python

In [2]:
import pandas as pd
import numpy as np

import os

import mysql.connector as msql
from mysql.connector import Error
import base64

from PIL import Image
import requests
from io import BytesIO

## Reading Data

In [3]:
url = "https://raw.githubusercontent.com/leticiatavaresds/Trabalho_Final_BD_Spotify/main/Data/dados_spotify_cidades.csv"
df_Charts_Cities = pd.read_csv(url, index_col=False, delimiter = ',')
df_Charts_Cities.rename(columns={"Track_id": "Song_id"}, inplace= True)
df_Charts_Cities.rename(columns={"City": "Chart"}, inplace= True)

df_Charts_Cities["ID_Chart"] = df_Charts_Cities.Chart.str.lower().str.replace(" ", "_").str.replace("local_pulse", "lp")
df_Charts_Cities["ID_Chart"] = df_Charts_Cities.ID_Chart.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode("utf-8")

df_Charts_Cities["Type"]= np.where(df_Charts_Cities['Chart'].str.contains('Local Pulse'), "Local Pulse", "Normal")
df_Charts_Cities["Cidade"] = df_Charts_Cities.Chart.str.replace("Local Pulse", "")

url = "https://raw.githubusercontent.com/leticiatavaresds/Trabalho_Final_BD_Spotify/main/Data/data_songs.csv"
df_Songs = pd.read_csv(url, index_col=False, delimiter = ',')

url = "https://raw.githubusercontent.com/leticiatavaresds/Trabalho_Final_BD_Spotify/main/Data/IBGE_cidades.csv"
df_Cities = pd.read_csv(url, index_col=False, delimiter = ',')

df_Charts_Songs = pd.merge(df_Charts_Cities,df_Songs, on = 'Song_id')
df_Charts_Songs = pd.merge(df_Charts_Songs,df_Cities[["Cidade", "Código_Cidade"]], on = 'Cidade')

## Creating Dataframes

In [4]:
# Chart Table
df_table_chart = df_Charts_Songs[['ID_Chart', 'Código_Cidade', 'Type']].drop_duplicates().reset_index(drop = True)
df_table_chart.rename(columns={"ID_Chart": "ID",
                               "Type": "Tipo"}, inplace= True)

df_table_chart.head(3)

,ID,Código_Cidade,Tipo
0,belo_horizonte,3106200,Normal
1,belem,1501402,Normal
2,campinas,3509502,Normal


In [5]:
# Cities Table
df_table_cities = df_Cities.copy()
df_table_cities.head(4)

,Código_Cidade,Cidade,Região
0,1302603,Manaus,Norte
1,1501402,Belém,Norte
2,2304400,Fortaleza,Nordeste
3,2611606,Recife,Nordeste


In [6]:
# Chart songs Table
df_table_chart_songs = df_Charts_Songs[['ID_Chart', 'Date', 'Song_id', 'Position']].reset_index(drop = True)
df_table_chart_songs.rename(columns={"ID_Chart": "ID_Ranking",
                               "Date": "Data",
                               "Song_id": "ID_Musica",
                               "Position": "Posição",}, inplace= True)
df_table_chart_songs.head(3)

,ID_Ranking,Data,ID_Musica,Posição
0,belo_horizonte,2022-02-03,0NBgyml7T0IvUNbXIkcSpH,1.0
1,belo_horizonte,2022-01-27,0NBgyml7T0IvUNbXIkcSpH,1.0
2,belo_horizonte,2022-01-20,0NBgyml7T0IvUNbXIkcSpH,1.0


In [7]:
## Album Songs Table
df_table_songs = df_Charts_Songs[['Song_id', 'Album_id', 'Song_Name', 'Song_Explicit', 'Song_popularity',
                                       'Song_danceability','Song_energy', 'Song_loudness',
                                        'Song_valence', 'Song_track_number'
                                       ]].drop_duplicates().reset_index(drop = True)

df_table_songs.rename(columns={"Song_id": "ID",
                               "Album_id": "ID_Album",
                               "Song_Name": "Nome",
                               "Song_Explicit": "Explicita",
                               "Song_popularity": "Popularidade",
                               "Song_danceability": "Danceabilidade",
                               "Song_energy": "Energia",
                               "Song_loudness": "Volume",
                               "Song_valence": "Valencia",
                               "Song_track_number": "Faixa"}, inplace= True)

df_table_songs.head(4)

,ID,ID_Album,Nome,Explicita,Popularidade,Danceabilidade,Energia,Volume,Valencia,Faixa
0,0NBgyml7T0IvUNbXIkcSpH,1XrBVMRctpsTGw189yBIsl,Malvadão 3,False,92,0.821,0.609,-6.541,0.516,1
1,5719MEdRQcH4c3phXqX7WN,6wM2qL8ddXGiEMPy44Wt8I,Mal Feito - Ao Vivo,False,89,0.732,0.831,-3.791,0.684,1
2,1c4htMHiolERc0YJvZNdlc,2ULXBNAz2pgBQmwc3qODUz,Vai Lá Em Casa Hoje,False,89,0.456,0.830,-3.646,0.619,1
3,6nkoWsFZeFt5dUdNmlIceh,1AebCIKwFh7upRidnha8yo,Bloqueado - Ao Vivo,False,79,0.828,0.721,-4.274,0.651,3


In [8]:
## Album Table
df_table_albums = df_Charts_Songs[['Album_id', 'Album_name', 'Album_release','Album_image']].drop_duplicates().reset_index(drop = True)
df_table_albums.rename(columns={"Album_id": "ID",
                               "Album_name": "Nome",
                               "Album_release": "Data",
                               "Album_image": "Imagem_capa"}, inplace= True)
df_table_albums.head(4)

,ID,Nome,Data,Imagem_capa
0,1XrBVMRctpsTGw189yBIsl,Malvadão 3,2021-11-30,https://i.scdn.co/image/ab67616d0000b273ead325...
1,6wM2qL8ddXGiEMPy44Wt8I,Mal Feito (Ao Vivo),2022-01-14,https://i.scdn.co/image/ab67616d0000b273054b55...
2,2ULXBNAz2pgBQmwc3qODUz,Vai Lá Em Casa Hoje,2021-10-22,https://i.scdn.co/image/ab67616d0000b273e3db1d...
3,1AebCIKwFh7upRidnha8yo,Buteco in Boston (Ao Vivo),2021-12-16,https://i.scdn.co/image/ab67616d0000b273fefe16...


In [9]:
## Artist Albums Table
df_table_artist_albums = df_Charts_Songs[['Artist_id', 'Album_id']].drop_duplicates().reset_index(drop = True)
df_table_artist_albums.rename(columns={"Album_id": "ID_Album",
                          "Artist_id": "ID_Artista"}, inplace= True)
df_table_artist_albums.head(4)

,ID_Artista,ID_Album
0,5YwzDz4RJfTiMHS4tdR5Lf,1XrBVMRctpsTGw189yBIsl
1,1LIuN7ov1IBQDdLsU83ojl,6wM2qL8ddXGiEMPy44Wt8I
2,3FlCKtwOEYQQYyGVEbxG2Z,2ULXBNAz2pgBQmwc3qODUz
3,7MiDcPa6UiV3In7lIM71IN,1AebCIKwFh7upRidnha8yo


In [10]:
## Artist Table
df_table_artists = df_Charts_Songs[['Artist_id', 'Artist_name', 'Artist_followers', 
                                    'Artist_popularity']].drop_duplicates().reset_index(drop = True)

df_table_artists.rename(columns={"Artist_id": "ID",
                               "Artist_name": "Nome",
                               "Artist_followers": "Seguidores",
                               "Artist_popularity": "Popularidade"}, inplace= True)
df_table_artists.head(4)

,ID,Nome,Seguidores,Popularidade
0,5YwzDz4RJfTiMHS4tdR5Lf,Xamã,1356506,82
1,1LIuN7ov1IBQDdLsU83ojl,Hugo & Guilherme,571090,78
2,3FlCKtwOEYQQYyGVEbxG2Z,George Henrique & Rodrigo,1262190,75
3,7MiDcPa6UiV3In7lIM71IN,Gusttavo Lima,15014715,84


In [11]:
## Artist Genders

artist_genders = []
df_artistis_genders = df_Charts_Songs[['Artist_id', 'Artist_genres']].drop_duplicates().reset_index(drop = True)

for i in range(len(df_artistis_genders)):
    
    list_genders = df_Charts_Songs["Artist_genres"][i].replace("[", "").replace("]", "").replace("'", "").split(", ")
    
    
    for gender in list_genders:
        artist_genders.append((df_artistis_genders["Artist_id"][i], gender))
        
df_table_artist_genders = pd.DataFrame(artist_genders, columns=['Artist_id', 'Gender'])
df_table_artist_genders.rename(columns={"Artist_id": "ID_Artista",
                                    "Gender": "Genero",}, inplace= True)
df_table_artist_genders.head(4)

,ID_Artista,Genero
0,5YwzDz4RJfTiMHS4tdR5Lf,brazilian hip hop
1,5YwzDz4RJfTiMHS4tdR5Lf,funk carioca
2,5YwzDz4RJfTiMHS4tdR5Lf,trap brasileiro
3,1LIuN7ov1IBQDdLsU83ojl,brazilian hip hop


## Creating DataBase

In [12]:
# try:
#     conn = msql.connect(host='db4free.net', 
#                         user = "grupo_spotify",  
#                         password = "ufrjbdspotify",
#                         port: "3306")
# except Error as e:
#     print("Error while connecting to MySQL", e)

In [13]:
try:
    conn = msql.connect(host='db4free.net', 
                        user = "grupo_spotifybd",  
                        password = "ufrj_bd_spotify",
                        port = "3306")
except Error as e:
    print("Error while connecting to MySQL", e)

In [14]:
cursor = conn.cursor()

## Creating Tables

In [15]:
database = "Ranking_Cidades_Brasileiras"

In [16]:
cursor.execute("USE {database};".format(database = database))

#### Ranking Table

In [17]:
table = "Ranking"

cursor.execute('DROP TABLE IF EXISTS {table};'.format(table = table))

cursor.execute("""CREATE TABLE {table}(ID varchar(255) NOT NULL PRIMARY KEY,
                                        Codigo_Cidade int NOT NULL,
                                        Tipo varchar(255) NOT NULL)""".format(table = table))
for i,row in df_table_chart.iterrows():

    sql = "INSERT INTO {database}.{table} VALUES (%s,%s,%s)".format(database = database,table = table)
    cursor.execute(sql, tuple(row))
    conn.commit()

#### Artist Table

In [18]:
table = "Artista"

cursor.execute('DROP TABLE IF EXISTS {table};'.format(table = table))

cursor.execute("""CREATE TABLE {table}( ID VARCHAR(255) NOT NULL PRIMARY KEY,
                                        Nome VARCHAR(255) NOT NULL,
                                        Seguidores INT NOT NULL,
                                        Popularidade INT NOT NULL)""".format(table = table))
for i,row in df_table_artists.iterrows():

    sql = "INSERT INTO {database}.{table} VALUES (%s,%s,%s,%s)".format(database = database,table = table)
    cursor.execute(sql, tuple(row))
    conn.commit()

#### Album Table

In [19]:
table = "Album"

cursor.execute('DROP TABLE IF EXISTS {table};'.format(table = table))

cursor.execute("""CREATE TABLE {table}( ID VARCHAR(255) NOT NULL PRIMARY KEY,
                                        Nome VARCHAR(255) NOT NULL,   
                                        Data DATE,
                                        Imagem_capa MEDIUMBLOB NOT NULL)""".format(table = table))

for i,row in df_table_albums.iterrows():
    
    url = row[3]    
    img_bytes = requests.get(url).content
    data = row[0:3].tolist()
    data.append(img_bytes)
         
    try: 
        int(row[2])
        sql = "INSERT INTO {database}.{table} VALUES (%s,%s,NULL,%s)".format(database = database,table = table)
        data.remove(row[2])   
        cursor.execute(sql, tuple(data))
        
    except :
        sql = "INSERT INTO {database}.{table} VALUES (%s,%s,%s,%s)".format(database = database,table = table)
        cursor.execute(sql, tuple(data))
        
    conn.commit()

#### Song Table

In [20]:
df_table_songs.Explicita = df_table_songs.Explicita.astype(int)

table = "Música"

cursor.execute('DROP TABLE IF EXISTS {table};'.format(table = table))

cursor.execute("""CREATE TABLE {table}( ID VARCHAR(255) NOT NULL PRIMARY KEY,
                                        ID_Album VARCHAR(255) NOT NULL,
                                        Nome VARCHAR(255) NOT NULL,
                                        Faixa INT NOT NULL,
                                        Explicita TINYINT NOT NULL,
                                        Popularidade INT NOT NULL,
                                        Danceabilidade FLOAT(3) NOT NULL,
                                        Energia FLOAT(3) NOT NULL,
                                        Volume FLOAT(3) NOT NULL,
                                        Valencia FLOAT(3) NOT NULL,
                                        FOREIGN KEY (ID_Album) REFERENCES Album(ID))""".format(table = table))
for i,row in df_table_songs.iterrows():

    sql = "INSERT INTO {database}.{table} VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)".format(database = database,table = table)
    cursor.execute(sql, tuple(row))
    conn.commit()

#### City Table

In [21]:
df_table_songs.Explicita = df_table_songs.Explicita.astype(int)

table = "Cidade"

cursor.execute('DROP TABLE IF EXISTS {table};'.format(table = table))

cursor.execute("""CREATE TABLE {table}( ID INT NOT NULL PRIMARY KEY,
                                        Cidade VARCHAR(255) NOT NULL,
                                        Região VARCHAR(255) NOT NULL)""".format(table = table))
for i,row in df_table_cities.iterrows():

    sql = "INSERT INTO {database}.{table} VALUES (%s,%s,%s)".format(database = database,table = table)
    cursor.execute(sql, tuple(row))
    conn.commit()

#### Gender Table

In [22]:
df_table_songs.Explicita = df_table_songs.Explicita.astype(int)

table = "Gênero"

cursor.execute('DROP TABLE IF EXISTS {table};'.format(table = table))

cursor.execute("""CREATE TABLE {table}( ID_Artista VARCHAR(255) NOT NULL,
                                        Gênero VARCHAR(255),
                                        FOREIGN KEY (ID_Artista) REFERENCES Artista(ID))""".format(table = table))

for i,row in df_table_artist_genders.iterrows():

    sql = "INSERT INTO {database}.{table} VALUES (%s,%s)".format(database = database,table = table)
    cursor.execute(sql, tuple(row))
    conn.commit()

#### Artist's Albums

In [23]:
df_table_songs.Explicita = df_table_songs.Explicita.astype(int)

table = "Performa"

cursor.execute('DROP TABLE IF EXISTS {table};'.format(table = table))

cursor.execute("""CREATE TABLE {table}( ID_Artista VARCHAR(255) NOT NULL,
                                        ID_Album VARCHAR(255) NOT NULL,
                                        FOREIGN KEY (ID_Artista) REFERENCES Artista(ID),
                                        FOREIGN KEY (ID_Album) REFERENCES Album(ID))""".format(table = table))

for i,row in df_table_artist_albums.iterrows():

    sql = "INSERT INTO {database}.{table} VALUES (%s,%s)".format(database = database,table = table)
    cursor.execute(sql, tuple(row))
    conn.commit()

#### Ranking's Songs

In [ ]:
table = "Performa"

cursor.execute('DROP TABLE IF EXISTS {table};'.format(table = table))

cursor.execute("""CREATE TABLE {table}( ID_Ranking VARCHAR(255) NOT NULL,
                                        Data DATE NOT NULL,
                                        ID_Música VARCHAR(255) NOT NULL,                                        
                                        Posição INT NOT NULL,
                                        FOREIGN KEY (ID_Ranking) REFERENCES Ranking(ID),
                                        FOREIGN KEY (ID_Música) REFERENCES Música(ID))""".format(table = table))

for i,row in df_table_chart_songs.iterrows():

    sql = "INSERT INTO {database}.{table} VALUES (%s,%s,%s,%s)".format(database = database,table = table)
    cursor.execute(sql, tuple(row))
    conn.commit()